## Code Appendix

### 4.1: Implement Decision Trees & 4.2: Implement a Random Forest
#### this is done in decision_tree_starter.py & is available in code index

### 4.3: Describe implementation details

#### 1. For categorical features I used one hot encoding to get binary values and for missing values I used the mode of column 
#### 2. Stopping criterion was based on the depth I wanted the tree to go. 
#### 3. By making/using the decision tree class in decision_tree_starter.py and train the decision trees with m random features and sampling with replacement. 
#### 4. Other than removing unnecessary features or experimentings with depth levels, I did not. 
#### 5. Random forest perhaps since its only one that works. 

### 4.4 Performance Evaluation
#### Kaggle Username: Jose Fernandez
#### Kaggle Spam Score: 0.735
#### Kaggle Titanic Score: N/A
#### below are the accuracy scores for the spam and titanic datasets(4 spam scores & 2 titanic scores)

In [227]:
import io
from collections import Counter
import numpy as np
import scipy.io
import sklearn.model_selection
import sklearn.tree
from numpy import genfromtxt
from scipy import stats
from sklearn.base import BaseEstimator, ClassifierMixin
import decision_tree_starter
eps = 1e-5 # a small number
def most_frequent(List):
    dict = {}
    count, itm = 0, ''
    for item in reversed(List):
        if not item:
            pass
    dict[item] = dict.get(item, 0) + 1
    if dict[item] >= count :
        count, itm = dict[item], item
    return(itm)


In [228]:
# load data
spam_data = scipy.io.loadmat("/Users/jose/Downloads/hw5/datasets/spam_data/spam_data.mat")
spam_train_data = spam_data['training_data']
spam_train_labels = spam_data['training_labels'].flatten()
spam_test_data = spam_data['test_data']
titanic_train_data = genfromtxt("/Users/jose/Downloads/hw5/datasets/titanic/titanic_training.csv",
    delimiter=',')
titanic_test_data = genfromtxt("/Users/jose/Downloads/hw5/datasets/titanic/titanic_testing_data.csv",
delimiter=',')
S_train, S_test, SY_train, SY_test = train_test_split(spam_train_data,
    spam_train_labels, test_size=0.2)
# shape of datasets
print(f"Shape of Spam Train: {spam_train_data.shape}")
print(f"Shape of Spam Train Labels: {spam_train_labels.shape}")
print(f"Shape of Spam Test: {spam_test_data.shape}")
print(f"Shape of Titanic Train: {titanic_train_data.shape}")
print(f"Shape of Titanic Labels: {titanic_test_data.shape}")

Shape of Spam Train: (5629, 32)
Shape of Spam Train Labels: (5629,)
Shape of Spam Test: (5400, 32)
Shape of Titanic Train: (1010, 10)
Shape of Titanic Labels: (301, 9)


In [229]:
# classify Spam.
spam_features = [
           "pain", "private", "bank", "money", "drug", "spam", "prescription",
               "creative",
"height", "featured", "differ", "width", "other", "energy",
               "business", "message",
           "buy", "click here", "instant", "limited time", "100%", "access",
               "guarantee",
           "offer", "re:", "<head>", "<title>", "<body>", "<html>", "#", "&", "volumes", "revision", "path",
            "memo", "planning", "pleased", "record", "out"
            ]
spam_tree = DecisionTree(max_depth = 5, feature_labels = spam_features)
# fitting, predicting, and evaluation.
spam_tree.fit(S_train, SY_train)
spam_predictions = spam_tree.predict(S_train)
spam_predictions_Y = spam_tree.predict(S_test)
spam_train_accuracy = sum(spam_predictions == SY_train) / len(spam_predictions)
spam_train_accuracy_Y = sum(spam_predictions_Y == SY_test) / len(spam_predictions_Y)
print(f"Our training accuracy for the Spam Training dataset is:{spam_train_accuracy}")
print(f"Our training accuracy for the Spam Testing dataset is:{spam_train_accuracy_Y}")

Our training accuracy for the Spam Training dataset is:0.7257383966244726
Our training accuracy for the Spam Testing dataset is:0.7291296625222025


In [230]:
# classify Spam using Random Forests.
params = {"max_depth": 2,
         "feature_labels": spam_features}
spam_tree_RF = RandomForest(params=params, m=2, n=200)
# No need to split. Proceed with fitting, predicting, and evaluation.
spam_tree_RF.fit(S_train, SY_train)
spam_predictions_RF = spam_tree_RF.predict(S_train)
spam_predictions_RF_Y = spam_tree_RF.predict(S_test)
spam_train_accuracy_RF = sum(spam_predictions_RF == SY_train) / len(spam_predictions_RF)
spam_train_accuracy_RF_Y = sum(spam_predictions_RF_Y == SY_test) / len(spam_predictions_RF_Y)
print(f"Our training accuracy for the Spam Training dataset using Random Forests is: {spam_train_accuracy_RF}")
print(f"Our training accuracy for the Spam Testing dataset using Random Forests is: {spam_train_accuracy_RF_Y}")

Our training accuracy for the Spam Training dataset using Random Forests is: 0.7255163224516988
Our training accuracy for the Spam Testing dataset using Random Forests is: 0.7291296625222025


In [231]:
# classify the Titanic Dataset.
# preprocessing automatically.
def preprocessingTitanic(data, train=True):
    if train:
        # Convert to df.
        df_columns = ["embarked", "cabin", "fare", "ticket", "parch", "sibsp","sex", "age", "pclass", "survived"]
        df = pd.DataFrame(data, columns = df_columns)
        # Handle NaN values. Remove columns with entirely NaN values. Replace others with mode’s of the column.
        MMM = np.array([])
        for i in df_columns:
            MMM = np.append(MMM, most_frequent(df[i].to_numpy()))
        df = df.drop(["cabin", "age", "pclass"], axis=1)
        surviving_columns = ["embarked", "fare", "ticket", "parch", "sibsp",
        "sex", "survived"]
        MMM = np.delete(np.delete((np.delete(MMM, 1)), 6), 6)
        for i, col in enumerate(surviving_columns):
            df[col] = df[col].fillna(MMM[i])
        # Convert back to column, return training and labels
        return df[["embarked", "fare", "ticket", "parch", "sibsp", "sex"]].to_numpy(), df[["survived"]].to_numpy()
    else:
        # Convert to df.
        df_columns = ["embarked", "cabin", "fare", "ticket", "parch", "sibsp","sex", "age", "pclass"]
        df = pd.DataFrame(data, columns = df_columns)
        # Handle NaN values. Remove columns with entirely NaN values. Replace others with mode’s of the column.
        MMM = np.array([])
        for i in df_columns:
            MMM = np.append(MMM, most_frequent(df[i].to_numpy()))
        df = df.drop(["cabin", "age", "pclass"], axis=1)
        surviving_columns = ["embarked", "fare", "ticket", "parch", "sibsp", "sex"]
        MMM = np.delete(np.delete((np.delete(MMM, 1)), 6), 6)
        for i, col in enumerate(surviving_columns):
            df[col] = df[col].fillna(MMM[i])
        # Convert back to column
        return df.to_numpy()

In [232]:
# process both of our datasets.
titanic_train_data, titanic_train_labels = preprocessingTitanic(titanic_train_data, True)
titanic_test_data = preprocessingTitanic(titanic_test_data, False)
titanic_train_labels = titanic_train_labels.flatten()
T_train, T_test, TY_train, TY_test = train_test_split(titanic_train_data, titanic_train_labels, test_size=0.2)
# Next, we fit these datasets to our model.
titanic_features = ["embarked", "fare", "ticket", "parch", "sibsp", "sex"]
titanic_tree = DecisionTree(max_depth = 5)

In [233]:
# fitting, predicting, and evaluation.
titanic_tree.fit(T_train, TY_train)
titanic_predictions = titanic_tree.predict(T_train)
titanic_predictions_Y = titanic_tree.predict(T_test)
titanic_train_accuracy = sum(titanic_predictions == TY_train) /len(titanic_predictions)
titanic_train_accuracy_Y = sum(titanic_predictions_Y == TY_test) /len(titanic_predictions_Y)
print(f"Our training accuracy for the Titanic Training dataset is:{titanic_train_accuracy}")
print(f"Our training accuracy for the Titanic Testing dataset is: {titanic_train_accuracy_Y}")

Our training accuracy for the Titanic Training dataset is:0.0
Our training accuracy for the Titanic Testing dataset is: 0.0


In [234]:
# Random forests classifier to examine Titanic dataset
titanic_params = {"max_depth": 2,
"feature_labels": titanic_features}
titanic_tree_RF = decision_tree_starter.RandomForest(params=titanic_params, m=2,
    n=200)
# fitting, predicting, and evaluation.
titanic_tree_RF.fit(T_train, TY_train)
titanic_predictions_RF = titanic_tree_RF.predict(T_train)
titanic_predictions_RF_Y = titanic_tree_RF.predict(T_test)
titanic_train_accuracy_RF = sum(titanic_predictions_RF == TY_train) /len(titanic_predictions_RF)
titanic_train_accuracy_RF_Y = sum(titanic_predictions_RF_Y == TY_test) /len(titanic_predictions_RF_Y)
print(f"Our training accuracy for the Titanic Training dataset using Random Forests is: {titanic_train_accuracy_RF}")
print(f"Our training accuracy for the Titanic Testing dataset using Random Forests is: {titanic_train_accuracy_RF_Y}")


TypeError: DecisionTreeClassifier.__init__() got an unexpected keyword argument 'feature_labels'

In [236]:
# Final Kaggle Predictions
def results_to_csv(file, name):
    file = file.astype(int)
    df = pd.DataFrame({'Category': file})
    df.index += 1
    df.to_csv(name, index_label="Id")

final_spam_prediction = spam_tree_RF.predict(spam_test_data)
results_to_csv(final_spam_prediction, "spam_submission.csv")

In [ ]:
final_titanic_prediction = titanic_tree_RF.predict(titanic_test_data)
results_to_csv(final_titanic_prediction, "titanic_submission.csv")